# Install/import packages

In [2]:
# install necessary packages
!pip install pymatgen
!pip install mp_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.0/809.0 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 58.5 MB/s eta 0:00:00
   ━━━━

In [3]:
# import necessary packages
import pymatgen
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.core.periodic_table import Element
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from mp_api.client import MPRester
from pymatgen.core import Structure
import numpy as np
import pandas as pd

# Define API key and structural descriptors

In [5]:
# Define API key for data extraction from the MPD
API_KEY = "mhz7IO0VGT4oZ7S8qu1pHsPL4zOQw7kx" # INSERT YOUR API KEY HERE

In [6]:
'''
Compute 5 structural descriptors including the following:
1. Lithium-Lithium Bonding (LLB)
2. Sublattice Bond Ionicity (SBI)
3. Anion Framework Coordination (AFC)
4. Li-Anion Separation Distance (LASD)
5. Li-Li Separation Distance (LLSD)
'''

def compute_descriptors(structure):

    results = {}

    # Initialize neighbor analysis tool
    cnn = CrystalNN()

    # Get Li sites
    li_sites = [site for site in structure if site.species_string == "Li"]

    # 1. Li-Li Bond Count (LLB)
    li_bonds = 0
    for li in li_sites:
        neighbors = cnn.get_nn_info(structure, structure.index(li))
        li_neighbors = [n for n in neighbors if n["site"].species_string == "Li"]
        li_bonds += len(li_neighbors)
    results["LLB"] = li_bonds / len(li_sites) if li_sites else 0

    # 2. Sublattice Bond Ionicity (SBI)
    ionicity_sum = 0
    for li in li_sites:
        neighbors = cnn.get_nn_info(structure, structure.index(li))
        for neighbor in neighbors:
            delta_X = abs(Element("Li").X - Element(neighbor["site"].species_string).X)
            ionicity_sum += delta_X
    results["SBI"] = ionicity_sum / len(structure) if len(structure) > 0 else 0

    # 3. Anion Framework Coordination (AFC)
    anions = [site for site in structure]
    total_coord = sum(len(cnn.get_nn_info(structure, structure.index(anion))) for anion in anions)
    results["AFC"] = total_coord / len(anions) if anions else 0

    # 4. Li-Anion Separation Distance (LASD)
    anion_sites = [site for site in structure]
    if li_sites and anion_sites:
        distances = [li.distance(anion) for li in li_sites for anion in anion_sites]
        results["LASD"] = np.mean(distances)
    else:
        results["LASD"] = 0

    # 5. Li-Li Separation Distance (LLSD)
    if len(li_sites) >= 2:
        distances = []
        for i in range(len(li_sites)):
            for j in range(i+1, len(li_sites)):
                distances.append(li_sites[i].distance(li_sites[j]))
        results["LLSD"] = np.mean(distances)
    else:
        results["LLSD"] = 0

    return results

# Retrieve parameters of all Li-based compounds from the Materials Project Database (MPD) for full analysis of potential new superionic materials

In [7]:
'''
Retrieve compound properties from the MPD to calculate 5 structural
descriptors including:
1. MPD ID
2. Chemical formula
3. Number of elements in compound
4. Band gap in eV
5. Thermodynamic stability in eV
6. Gibbs free energy in eV
'''

# Query Materials Project database for all Li-based compounds
with MPRester(API_KEY) as mpr:
    query = mpr.materials.summary.search(
        elements=["Li"],
        fields=["material_id", "structure", "nelements", "band_gap",
                "energy_above_hull", "formation_energy_per_atom"]
    )

Retrieving SummaryDoc documents:   0%|          | 0/21756 [00:00<?, ?it/s]

In [8]:
# Process all materials and compute descriptors
data = []
for doc in tqdm(query, total=len(query)):
    try:
        struct = doc.structure
        descriptors = compute_descriptors(struct)
        descriptors.update({
            "material_id": doc.material_id,
            "formula": struct.formula,
            "num_elements": doc.nelements,
            "bandgap": doc.band_gap,
            "energy_above_hull": doc.energy_above_hull,
            "formation_energy_per_atom": doc.formation_energy_per_atom
        })
        data.append(descriptors)
    except Exception as e:
        print(f"Error processing {doc.material_id}: {str(e)}")

  0%|          | 0/21756 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:4019: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r1 = _get_radius(structure[n])
/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:4021: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r2 = _get_radius(entry["site"])
/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:3933: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  nn_data = self.get_nn_data(structure, n)
100%|██████████| 21756/21756 [3:59:12<00:00,  1.52it/s]


In [14]:
# Create DataFrame and save to .csv file
Li_materials_df = pd.DataFrame(data)
#csv = df.to_csv("li_structural_descriptors.csv", index=False)

print(f"Successfully processed {len(data)}/{len(query)} materials")
print(Li_materials_df.head())

Successfully processed 21756/21756 materials
    LLB  SBI   AFC      LASD      LLSD material_id formula  num_elements  \
0  12.0  0.0  12.0  2.037129  3.055694  mp-1018134     Li3             1   
1  12.0  0.0  12.0  2.057606  3.086410  mp-1063005     Li3             1   
2  12.0  0.0  12.0  1.545429  3.090857    mp-10173     Li2             1   
3  12.0  0.0  12.0  2.862107  3.816142   mp-976411     Li4             1   
4   6.0  0.0   6.0  2.045836  2.727781   mp-604313     Li4             1   

   bandgap  energy_above_hull  formation_energy_per_atom  
0      0.0           0.000000                   0.000000  
1      0.0           0.015671                   0.015671  
2      0.0           0.005988                   0.005988  
3      0.0           0.005889                   0.005889  
4      0.0           0.261799                   0.261799  


**Pre-filtering MPD Materials**

Many of the 20,000+ Li-based materials in the Materials Project Database can be pre-filtered based on the following parameters:

1. **Band gap**: Materials with a band gap lower than 1 eV can conduct electrons too easily and thus would not be effective ion conductors.
2. **Energy above hull**: Materials need to be thermodynamically stable to operate as an effective ion conductor, so $E_{hull}$ should equal 0 eV.
3. **Contains transition metals**: Previous studies have shown that Li-based materials without transition metals typically do not operate as effective ion conductors.

This filtering process decreases the size of our dataset from 21,756 to 452 Li-based materials.

In [15]:
# define list of transition metals
transition_metals = [
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",  # 3rd period
    "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",  # 4th period
    "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",        # 5th period
    "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn"        # 6th period
]

# filter dataframe of Li-based materials based on parameters listed above
for material_index in range(len(Li_materials_df)):
  if (Li_materials_df.bandgap[material_index] < 1) or \
     (not Li_materials_df.energy_above_hull[material_index] == 0) or \
     (any(metal in Li_materials_df.formula[material_index] for metal in
         transition_metals)):
      Li_materials_df = Li_materials_df.drop(index=material_index)

In [16]:
# Export values to .csv file
csv = Li_materials_df.to_csv("li_structural_descriptors_filtered.csv", index=False)
from google.colab import files
files.download('li_structural_descriptors_filtered.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>